# Table of Contents

1. [Importing Necessary Dependencies](#importing-necessary-dependencies)
2. [Loading the Datasets](#loading-the-datasets)
3. [Defining Helper Functions](#defining-helper-functions)
4. [Data Preprocessing](#data-preprocessing)
5. [Model Training](#model-training)
6. [Model Evaluation](#model-evaluation)

# Importing Necessary Dependencies

In [19]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, learning_curve
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.utils import class_weight
from xgboost import XGBClassifier

# Loading the Datasets

In [20]:
X_train = pd.read_parquet('../data/processed/X_train.parquet')
X_test = pd.read_parquet('../data/processed/X_test.parquet')
y_train = pd.read_parquet('../data/processed/y_train.parquet')
y_test = pd.read_parquet('../data/processed/y_test.parquet')

In [21]:
X_train.drop(columns="code_snippet", inplace=True)
X_test.drop(columns="code_snippet", inplace=True)

In [22]:
X_train.head()

,loc,num_args,num_returns,num_variables,num_function_calls,has_decorators,uses_globals,is_recursive,estimated_difficulty,estimated_bugs,has_docstring,docstring_length,num_comments,name_length,is_name_well_formed,bad_variable_names_count,max_return_length,estimated_complexity
0,2.0,1.0,0,0,1,False,False,False,0.000000,0.000000,False,0,0.0,46.0,True,0.0,0.0,1.0
1,13.0,1.0,0,7,6,False,False,False,1.250000,0.016610,False,0,0.0,13.0,True,0.0,0.0,1.0
2,4.0,1.0,0,0,2,False,False,False,0.000000,0.000000,False,0,1.0,16.0,True,0.0,0.0,1.0
3,67.0,2.0,2,28,38,False,False,False,4.442308,0.084073,True,3,6.0,14.0,True,11.0,125.0,9.0
4,6.0,2.0,0,0,5,True,False,False,0.000000,0.000000,False,0,0.0,35.0,False,0.0,0.0,1.0


In [23]:
y_train.head()

,quality
0,good
1,bad
2,good
3,bad
4,good


In [24]:
X_train = X_train[~X_train.duplicated()]
X_test = X_test[~X_test.duplicated()]

train_keep_indices = X_train.index
test_keep_indices = X_test.index

# Filter y_train and y_test to match the deduplicated X's
y_train = y_train.loc[train_keep_indices]
y_test = y_test.loc[test_keep_indices]

In [25]:
X_train.describe()

,loc,num_args,num_returns,num_variables,num_function_calls,estimated_difficulty,estimated_bugs,docstring_length,num_comments,name_length,bad_variable_names_count,max_return_length,estimated_complexity
count,176967.000000,176967.000000,176967.000000,176967.000000,176967.000000,176967.000000,176967.000000,176967.000000,176967.000000,176967.000000,176967.000000,176967.000000,176967.000000
mean,18.513926,1.831053,0.769449,3.499698,7.981974,0.767141,0.007568,2.052371,1.116768,18.922584,0.957958,20.059226,2.541084
std,25.612529,1.665070,1.197201,5.172238,10.305177,1.364084,0.025296,9.490253,3.185020,11.655740,2.129401,62.775364,3.502940
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
25%,5.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,1.000000
50%,11.000000,1.000000,1.000000,2.000000,5.000000,0.500000,0.000667,0.000000,0.000000,17.000000,0.000000,1.000000,1.000000
75%,22.000000,2.000000,1.000000,5.000000,10.000000,1.000000,0.005170,0.000000,1.000000,26.000000,1.000000,26.000000,3.000000
max,858.000000,55.000000,65.000000,226.000000,503.000000,56.875000,1.448993,449.000000,150.000000,295.000000,216.000000,12370.000000,182.000000


In [26]:
y_train.describe()

,quality
count,176967
unique,2
top,bad
freq,108560


In [27]:
X_train.dtypes

loc                         float64
num_args                    float64
num_returns                   int64
num_variables                 int64
num_function_calls            int64
has_decorators                 bool
uses_globals                   bool
is_recursive                   bool
estimated_difficulty        float64
estimated_bugs              float64
has_docstring                  bool
docstring_length              int64
num_comments                float64
name_length                 float64
is_name_well_formed            bool
bad_variable_names_count    float64
max_return_length           float64
estimated_complexity        float64
dtype: object

In [28]:
y_train.dtypes

quality    category
dtype: object

# Defining Helper Functions

In [29]:
def plot_learning_curve(model, X, y, cv=5, title="Learning Curve"):
	train_sizes, train_scores, val_scores = learning_curve(
		model, X, y, cv=cv, scoring='accuracy',
		train_sizes=np.linspace(0.1, 1.0, 10),
		n_jobs=-1
	)
	
	plt.figure(figsize=(10, 6))
	plt.plot(
		train_sizes, np.mean(train_scores, axis=1), label='Training Score'
	)
	plt.plot(
		train_sizes, np.mean(val_scores, axis=1), label='Validation Score'
	)
	plt.fill_between(
		train_sizes, np.mean(train_scores, axis=1) - np.std(train_scores, axis=1),
		np.mean(train_scores, axis=1) + np.std(train_scores, axis=1), alpha=0.1
	)
	plt.fill_between(
		train_sizes, np.mean(val_scores, axis=1) - np.std(val_scores, axis=1),
		np.mean(val_scores, axis=1) + np.std(val_scores, axis=1), alpha=0.1
	)
	
	plt.title(title)
	plt.xlabel("Training Examples")
	plt.ylabel("Accuracy Score")
	plt.legend()
	plt.grid()
	plt.show()

# Data Preprocessing

## Converting Boolean Columns to Numerical (0, 1)

In [30]:
bool_cols = X_train.select_dtypes(include='bool').columns
X_train[bool_cols] = X_train[bool_cols].astype(int)
X_test[bool_cols] = X_test[bool_cols].astype(int)

## Scaling the Data

### Identifying numerical columns (excluding boolean/binary columns)

In [31]:
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
numerical_cols = [col for col in numerical_cols if col not in bool_cols]

### Scaling

In [32]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), numerical_cols),
        ('bool', 'passthrough', bool_cols)  # Skip scaling for boolean columns
    ],
    remainder='passthrough'  # This will handle any other columns if they exist
)

X_train_scaled = preprocessor.fit_transform(X_train)
X_test_scaled = preprocessor.transform(X_test)

### Converting to DataFrame to maintain column names

In [33]:
feature_names = numerical_cols + list(bool_cols)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_names)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_names)

## Encoding the Target Variable

In [34]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train.squeeze())

## Calculating Class Weights
This step is optional, but can help balance the dataset and improve the model's performance.

In [35]:
classes = np.unique(y_train_encoded)
weights = class_weight.compute_sample_weight('balanced', y_train_encoded)
weights

array([1.29348605, 0.8150654 , 1.29348605, ..., 0.8150654 , 0.8150654 ,
       0.8150654 ], shape=(176967,))

# Model Training

In [ ]:
model = XGBClassifier(
	n_estimators=150,
	max_depth=30,
	reg_alpha=0.1,
    reg_lambda=0.1,
	learning_rate=0.05,
	subsample=0.9,
	colsample_bytree=0.9,
	random_state=42,
	objective='binary:hinge',
	eval_metric='logloss',
	n_jobs=-1
)

model.fit(
	X_train_scaled,
	y_train_encoded,
	verbose=True
)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=40, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=-1,
              num_parallel_tree=None, ...)

# Model Evaluation

In [70]:
y_pred_encoded = model.predict(X_test_scaled)
y_pred = label_encoder.inverse_transform(y_pred_encoded)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         bad       0.84      0.86      0.85     28032
        good       0.80      0.76      0.78     19579

    accuracy                           0.82     47611
   macro avg       0.82      0.81      0.81     47611
weighted avg       0.82      0.82      0.82     47611



In [64]:
scores = cross_val_score(model, X_train_scaled, y_train_encoded, cv=5)
print(f"Cross-validation scores: {scores}")
print(f"Mean cross-validation score: {scores.mean()}")

Cross-validation scores: [0.80852687 0.80756625 0.80566779 0.80716526 0.80126014]
Mean cross-validation score: 0.8060372610791788


In [ ]:
xgb.plot_importance(model)
plt.show()

In [ ]:
plot_learning_curve(model, X_train_scaled, y_train_encoded, cv=3, title="XGBoost Learning Curve")